In [1]:
import ray.tune as tune
from ray.rllib.algorithms.ppo import PPOConfig
import ray
from matplotlib import pyplot as plt
import torch
from torch import _dynamo
from ray.air import session
import numpy as np

torch._dynamo.allow_in_graph(torch.distributions.kl.kl_divergence)
torch._dynamo.disallow_in_graph(torch.distributions.kl.kl_divergence)

# Set up a ppo config that corresponds to our atari-ppo.yaml
config = PPOConfig().rl_module(_enable_rl_module_api=True).training(_enable_learner_api=True)
config.environment("ALE/Breakout-v5", clip_rewards=True)
config.training( 
    train_batch_size= 5000,
    sgd_minibatch_size= 500,
    num_sgd_iter=10,
    vf_loss_coeff=0.01,
    lambda_=0.95,
    kl_coeff=0.5,
    clip_param=0.1,
    )
config.rollouts(
    num_rollout_workers=7, 
    num_envs_per_worker=16,
    batch_mode="truncate_episodes",
    observation_filter="NoFilter",
    )


2023-06-06 23:09:24,807	WARNING algorithm_config.py:2540 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


In [2]:
config.framework(
    torch_compile_worker=tune.grid_search([True, False]), 
    torch_compile_learner=tune.grid_search([True, False]),
)
config.resources(num_gpus_per_learner_worker=1)

run_config= ray.air.RunConfig(
    stop={"training_iteration": 1},
    )
# Only execute one trial at a time
tune_config = tune.TuneConfig(num_samples=1, max_concurrent_trials=1)


def trainable(config):
    cfg = PPOConfig.from_dict(config)
    algo = cfg.build()
    # Warm up
    algo.step()

    training_iter_times_ms = []

    # Train for 5 steps to get an estimate
    for x in range(10):
        results_dict = algo.step()  
        training_iter_time_ms = results_dict["timers"]["training_iteration_time_ms"]
        training_iter_times_ms.append(training_iter_time_ms)
    
    mean_training_iter_time_ms = np.mean(training_iter_times_ms)
        

    session.report({"mean_training_iter_time_ms": mean_training_iter_time_ms})  # Send the score to Tune.

tuner = tune.Tuner(
    tune.with_resources(trainable, tune.PlacementGroupFactory(
        # 1 Learner Worker should have one CPU and one GPU
        # 7 Rollout Workers should have one GPU
        [{"CPU": 1, "GPU": 1},
         {"CPU": 1},
         {"CPU": 1},
         {"CPU": 1},
         {"CPU": 1},
         {"CPU": 1},
         {"CPU": 1},
         {"CPU": 1},
        ]
    )),
    run_config=run_config,
    tune_config=tune_config,
    param_space=config,
)

results = tuner.fit()

(ImplicitFunc pid=148584) Install gputil for GPU system monitoring.
(trainable pid=148584) 2023-06-06 23:09:40,788	WARNING algorithm_config.py:2540 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(trainable pid=148584) A.L.E: Arcade Learning Environment (version 0.8.0+919230b)
(trainable pid=148584) [Powered by Stella]
(trainable pid=148584) /tmp/ray/session_2023-06-06_21-36-15_740974_2985/runtime_resources/py_modules_files/_ray_pkg_4498fcf18ec63924/ray/rllib/algorithms/algorithm.py:441: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::Deprec

In [3]:
df = results.get_dataframe()